In [2]:
import os

CACHE_DIR = r"C:\Users\vikto\OneDrive\Bureau\Cache"


In [3]:
import requests

seasons = [str(s) for s in range(2022,2026)]


In [4]:
base_url = "https://www.hockey-reference.com"
schedule_links = [f"{base_url}/leagues/NHL_{season}_games.html" for season in seasons]

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36 Edg/138.0.0.0",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.google.com/"
}

import time
import hashlib

# Takes URL --> file name for caching
def url_to_file(url):
    return hashlib.md5(url.encode('utf-8')).hexdigest() + '.html'

# Function to get html from a page. Checks cache first, else fetches page then caches
def get_page(url, headers=None):

    file = url_to_file(url)
    filename = os.path.join(CACHE_DIR, file)

    if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as f:
            print('File already cached!')
            return f.read()

    print(f"File not cached --> Fetching {url}...")
    page = requests.get(url, headers)
    time.sleep(3.1)
    if page.status_code == 200:
        print(f"Successfully fetched page, now caching...")
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(page.text)
            time.sleep(0.4)
            print(f"File Cached. Excellent. Moving right along...")
        return page.text
    else:
        print(f"Couldn't download page. Status code: {page.status_code}")
        print(f"Retry-after: {page.headers.get('Retry-After')}")
        return None







In [ ]:
import pandas as pd
from pandas.errors import SettingWithCopyWarning
import warnings
warnings.filterwarnings("ignore", category=SettingWithCopyWarning)
from io import StringIO
from bs4 import BeautifulSoup
import time

all_games = []
all_box_scores_links = []

# Grabs data from each season's page (including box scores links for each game)
for season in schedule_links:
    data = get_page(season, headers)
    soup = BeautifulSoup(data)
    data = StringIO(data)
    games = pd.read_html(data, match='NHL Regular Season Schedule')
    games = games[0]              # Dataframe
    all_games.append(games)
    box_scores_table = soup.find(id='games')  # Get table for box scores links
    box_scores_links = [l.get("href") for l in box_scores_table.find_all("a")]    # Get all links
    box_scores_links = [f"{base_url}{l}" for l in box_scores_links if l and "boxscores/" in l]   # Get absolute URLs
    all_box_scores_links.extend(box_scores_links)
    time.sleep(3)

games = pd.concat(all_games)

In [ ]:
|games = games.drop(['LOG', 'Unnamed: 6', 'Notes'], axis=1)    # dropping non-relevant columns
games['Home Team Win'] = games['G.1'] > games['G']     # creating win column
games['Home Team Win'] = games['Home Team Win'].astype(int)   # Converting all True/False to 1/0
games.rename(columns={'G' : 'Visitor G', 'G.1' : 'Home G'}, inplace=True)


In [ ]:
games

In [ ]:
#all_box_scores_links
print(f"Length of box_scores_links: {len(all_box_scores_links)}")
print(f"Length of games: {len(games)}")

In [ ]:
games.head()


In [ ]:
# Cell for testing purposes
from Data.team_abbreviations import team_map

all_scores = []
count = 0

# Get box score data for each game
for link in all_box_scores_links:
    data = get_page(link)
    soup = BeautifulSoup(data, 'html.parser')

    box = soup.find('div', class_='scorebox')    # Finding scorebox to get names
    teams = box.find_all('strong')
    visitors = teams[0].text.strip()             # extracting visitor team's name
    home = teams[1].text.strip()                 # extracting home team's name

    visitors = team_map.get(visitors)            # Mapping to abbreviation
    home = team_map.get(home)

    data = StringIO(data)

    # Getting scorebox tables
    v_table = pd.read_html(data, attrs={'id' : f'{visitors}_skaters'}, header=1)[0]
    h_table = pd.read_html(data, attrs={'id' : f'{home}_skaters'}, header=1)[0]

    # Find row where the player name/index contains "TOTAL" and get the last one
    v_stats_table = v_table[v_table.iloc[:, 1].str.contains('TOTAL', na=False)].iloc[[-1]]
    h_stats_table = h_table[h_table.iloc[:, 1].str.contains('TOTAL', na=False)].iloc[[-1]]

    # Only keeping relevant columns
    v_stats_table = v_stats_table[['PIM', 'S', 'S%']]
    h_stats_table = h_stats_table[['PIM', 'S', 'S%']]

    # Renaming for processing later
    v_stats_table.rename(columns={'PIM' : 'Visitors PIM', 'S' : 'Visitors S', 'S%' : 'Visitors S%'}, inplace=True)
    h_stats_table.rename(columns={'PIM' : 'Home PIM', 'S' : 'Home S', 'S%' : 'Home S%'}, inplace=True)

    # Resetting indices for concatenating
    v_stats_table.reset_index(drop=True, inplace=True)
    h_stats_table.reset_index(drop=True, inplace=True)

    # Concatenating both tables
    final_table = pd.concat([v_stats_table, h_stats_table], axis=1)

    all_scores.append(final_table)
    count = count + 1
    print(count)




In [ ]:
print(len(all_scores))
scores = pd.concat(all_scores)
scores

In [ ]:
# Combine & Renaming columns
combined_games = pd.concat([games.reset_index(drop=True), scores.reset_index(drop=True)], axis=1)
combined_games.rename(columns={'Visitor' : 'Opponent', 'Visitor G' : 'GA', 'Visitors S' : 'SA', 'Visitors S%' : 'SA%'}, inplace=True)
combined_games.rename(columns={'Visitors PIM' : 'Opponent PIM', 'Home G' : 'G', 'Home Team Win' : 'Win/Loss', 'Home PIM' : 'PIM', 'Home S' : 'S', 'Home S%': 'S%'}, inplace=True)

combined_games

In [ ]:
all_data = []

# Get goalie data for each game
for link in all_box_scores_links:
    data = get_page(link, headers=headers)
    data = StringIO(data)
    goalie_stats = pd.read_html(data, match='Goalies Table', header=1)   # List of tables (df)
    v_goalie_stats = goalie_stats[0]   # First table is visitors
    h_goalie_stats = goalie_stats[1]   # Second table is home


    # Getting team's primary goalie
    v_goalie_stats = v_goalie_stats[v_goalie_stats['Rk'] == 1]
    h_goalie_stats = h_goalie_stats[h_goalie_stats['Rk'] == 1]

    # Only keeping relevant columns
    v_goalie_stats = v_goalie_stats[['SV%']]
    h_goalie_stats = h_goalie_stats[['SV%']]

    v_goalie_stats.rename(columns={'SV%' : 'Opponent SV%'}, inplace=True)

    goalie_stats = pd.concat([h_goalie_stats, v_goalie_stats], axis=1)

    all_data.append(goalie_stats)




In [ ]:
# Adding goalie stats to main dataframe
goalie_s = pd.concat(all_data)
all_matches = pd.concat([combined_games.reset_index(drop=True), goalie_s.reset_index(drop=True)], axis=1)
all_matches

In [ ]:
# Changing columns order for clarity
wanted_order = ['Date', 'Time', 'Home', 'Opponent', 'Att.', 'G', 'GA', 'S', 'S%', 'SV%', 'PIM', 'SA', 'SA%', 'Opponent SV%', 'Opponent PIM', 'Win/Loss']

all_matches = all_matches[wanted_order]
all_matches

In [ ]:
# Re-organizing the Dataframe to be from a team's perspective rather than 1 row/game

home = all_matches.copy()
home.rename(columns={'Home' : 'Team', 'Win/Loss' : 'Result'}, inplace=True)
home['venue'] = 'Home'

away = all_matches.copy()
away['Team'] = away['Opponent']
away['Opponent'] = away['Home']
away['venue'] = 'Away'

# Flip home team/away stats to be from the away team's perspective
away['G'], away['GA'] = away['GA'], away['G']
away['S'], away['SA'] = away['SA'], away['S']
away['S%'], away['SA%'] = away['SA%'], away['S%']
away['SV%'], away['Opponent SV%'] = away['Opponent SV%'], away['SV%']
away['PIM'], away['Opponent PIM'] = away['Opponent PIM'], away['PIM']
away['Win/Loss'] = away['Win/Loss'].apply(lambda x: 1 - x)
away.rename(columns={'Win/Loss' : 'Result'}, inplace=True)
home



In [ ]:
# Cleaning / Rearranging data for ML
combined = pd.concat([home, away], ignore_index=True)
combined = combined.drop(columns=['Home'])
combined = combined.sort_values(by=['Team']).reset_index(drop=True)
combined

In [ ]:
# Moving venue column (for readability)

columns = combined.columns.tolist()
columns.remove('venue')
columns.insert(columns.index('Opponent') + 1, 'venue')

combined = combined[columns]
combined


In [ ]:
games['Date'] = pd.to_datetime(games['Date'])

In [ ]:
combined_sorted = combined.sort_values(by=['Team', 'Date']).reset_index(drop=True)
combined_sorted

In [ ]:
# Saving as CSV
combined.to_csv('data.csv')

In [ ]:
import pandas as pd
data = pd.read_csv('games.csv')
data

In [ ]:

from bs4 import Comment

from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from Data.team_abbreviations import team_map
from io import StringIO

edge_driver_path = r"C:\Users\vikto\Downloads\edgedriver_win64\msedgedriver.exe"
service = EdgeService(executable_path=edge_driver_path)
driver = webdriver.Edge(service=service)


all_adv = []
counter = 0

for link in all_box_scores_links:

    edge_driver_path = r"C:\Users\vikto\Downloads\edgedriver_win64\msedgedriver.exe"
    service = EdgeService(executable_path=edge_driver_path)
    driver = webdriver.Edge(service=service)

    data = get_page(link)
    soup = BeautifulSoup(data, 'html.parser')

    box = soup.find('div', class_='scorebox')    # Finding scorebox to get names
    teams = box.find_all('strong')
    visitors = teams[0].text.strip()             # extracting visitor team's name
    home = teams[1].text.strip()                 # extracting home team's name

    visitors = team_map.get(visitors)            # Mapping to abbreviation
    home = team_map.get(home)

    url = link
    driver.get(url)
    wait = WebDriverWait(driver, 2)

    vis = driver.find_element(By.ID, f'{visitors}_adv_ALLAll')
    home = driver.find_element(By.ID, f'{home}_adv_ALLAll')

    vis_table = vis.get_attribute('outerHTML')
    home_table = home.get_attribute('outerHTML')

    vis_table = StringIO(vis_table)
    home_table = StringIO(home_table)

    vis_df = pd.read_html(vis_table)[0]
    home_df = pd.read_html(home_table)[0]

    vis_df = vis_df[['SAT‑F', 'SAT‑A', 'CF%', 'oZS%']]
    home_df = home_df[['SAT‑F', 'SAT‑A', 'CF%', 'oZS%']]

    vis_df.rename(columns={'SAT‑F' : 'Opponent SAT‑F', 'SAT‑A' : 'Opponent SAT‑A', 'CF%' : 'Opponent CF%', 'oZS%' : 'Opponent oZS%'}, inplace=True)

    final = pd.concat([vis_df, home_df], axis=1)

    all_adv.append(final)
    counter = counter + 1
    print(counter)
    driver.quit()





In [ ]:
data

In [ ]:
data_sorted = data.sort_values(by=['Date', 'Time', 'Team']).reset_index(drop=True)
data_sorted

In [5]:
url = 'https://www.naturalstattrick.com/games.php?fromseason=20212022&thruseason=20232024&stype=2&sit=all&loc=B&team=All&rate=n'

import pandas as pd
from io import StringIO

data = get_page(url)
data = StringIO(data)

table1 = pd.read_html(data, attrs={'id' : 'teams'})[0]
table1

File already cached!


,Game,Team,Unnamed: 2,TOI,CF,CA,CF%,FF,FA,FF%,...,LDSF%,LDGF,LDGA,LDGF%,LDSH%,LDSV%,SH%,SV%,PDO,Attendance
0,"2021-10-12 - Penguins 6, Lightning 2",Pittsburgh Penguins,Limited Report Full Report,60:00,58,55,51.33,47,40,54.02,...,42.86,1,0,100.00,11.11,100.00,17.14,92.86,1.100,19092
1,"2021-10-12 - Penguins 6, Lightning 2",Tampa Bay Lightning,Limited Report Full Report,60:00,55,58,48.67,40,47,45.98,...,57.14,0,1,0.00,0.00,88.89,7.14,82.86,0.900,19092
2,"2021-10-12 - Kraken 3, Golden Knights 4",Seattle Kraken,Limited Report Full Report,60:00,63,58,52.07,44,47,48.35,...,50.00,1,0,100.00,6.67,100.00,9.68,86.67,0.963,18431
3,"2021-10-12 - Kraken 3, Golden Knights 4",Vegas Golden Knights,Limited Report Full Report,60:00,58,63,47.93,47,44,51.65,...,50.00,0,1,0.00,0.00,93.33,13.33,90.32,1.037,18431
4,"2021-10-13 - Canadiens 1, Maple Leafs 2",Montreal Canadiens,Limited Report Full Report,60:00,55,64,46.22,41,51,44.57,...,57.89,0,0,-,0.00,100.00,3.13,93.33,0.965,18493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7865,"2024-04-18 - Oilers 1, Avalanche 5",Edmonton Oilers,Limited Report Full Report,60:00,53,58,47.75,41,41,50.00,...,45.00,0,1,0.00,0.00,90.91,3.85,81.48,0.853,18129
7866,"2024-04-18 - Ducks 4, Golden Knights 1",Anaheim Ducks,Limited Report Full Report,60:00,47,67,41.23,31,45,40.79,...,50.00,1,1,50.00,8.33,91.67,17.39,96.97,1.144,18109
7867,"2024-04-18 - Ducks 4, Golden Knights 1",Vegas Golden Knights,Limited Report Full Report,60:00,67,47,58.77,45,31,59.21,...,50.00,1,1,50.00,8.33,91.67,3.03,82.61,0.856,18109
7868,"2024-04-18 - Blackhawks 4, Kings 5",Chicago Blackhawks,Limited Report Full Report,60:06,27,72,27.27,20,58,25.64,...,21.05,1,1,50.00,25.00,93.33,30.77,85.71,1.165,18145


In [ ]:
|url2 = 'https://www.naturalstattrick.com/games.php?fromseason=20242025&thruseason=20242025&stype=2&sit=all&loc=B&team=All&rate=n'

html = get_page(url2)
html = StringIO(html)

table2 = pd.read_html(html, attrs={'id' : 'teams'})[0]
table2

In [ ]:
all_data = pd.concat([table1, table2], ignore_index=True)
all_data

In [ ]:
def format_opponent(df):
    df = df.copy()
    df['Opponent'] = df.groupby('Game')['Team'].transform(lambda x: x[::-1].values)
    return df

formatted_data = format_opponent(all_data)
formatted_data

In [ ]:
formatted_data['Date'] = formatted_data['Game'].str.extract(r'(^\d{4}-\d{2}-\d{2})')  # extracting data from the Game column
formatted_data['Date'] = pd.to_datetime(formatted_data['Date'])
formatted_data.drop(columns=['Game', 'Unnamed: 2'], inplace=True)
formatted_data

In [ ]:
formatted_data['Result'] = formatted_data['GF'] > formatted_data['GA']
formatted_data['Result'] = formatted_data['Result'].astype(int)
formatted_data

In [ ]:
formatted_data.to_csv('newdata.csv')